In [ ]:
import pywt
import cv2
import seaborn as sns
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

print("All libraries installed successfully!")


In [ ]:
img = cv2.imread(r'D:\College\NCI\VScode\Supervised_image_classification\celebrity_faces\lionel_messi\_111066400_messi.jpg')
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray.shape

In [ ]:
plt.imshow(gray, cmap='gray')

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(r'D:\College\NCI\VScode\Supervised_image_classification\open_cv\haarcascades\haarcascade_eye.xml')

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
faces
eyes = eye_cascade.detectMultiScale(gray)
eyes

In [ ]:
for (x, y, w, h) in faces:
    cv2.rectangle(gray, (x, y), (x+w, y+h), (255, 0, 0), 2)

plt.imshow(gray, cmap='gray')
plt.axis('off')
plt.show()


In [ ]:

for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt

def get_croped_face(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print("Image not found:", image_path)
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi_color = img[y:y+h, x:x+w]
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 1:
            return roi_color

    return None

In [ ]:
original_image = cv2.imread(r'D:\College\NCI\VScode\Supervised_image_classification\celebrity_faces\virat_kohli\fc47090678.jpg')
plt.imshow(original_image)

In [ ]:
cropped_face = get_croped_face(r'D:\College\NCI\VScode\Supervised_image_classification\celebrity_faces\virat_kohli\fc47090678.jpg')
if isinstance(cropped_face, type(None)):
    print("No face detected.")
else:
    plt.imshow(cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
path_to_dataset = r'D:\College\NCI\VScode\Supervised_image_classification\celebrity_faces'
path_to_cr_data = r'D:\College\NCI\VScode\Supervised_image_classification\cropped_celebrity_faces'

In [ ]:
import os
img_dirs = []
for entry in os.scandir(path_to_dataset):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [ ]:
img_dirs

In [ ]:
import shutil
if os.path.exists(path_to_cr_data):
    shutil.rmtree(path_to_cr_data)
os.makedirs(path_to_cr_data)

In [ ]:
cropped_img_dir = []
celebrity_file_names_dict = {}

for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('\\')[-1]
    print("Processing images for celebrity:", celebrity_name)
    celebrity_file_names_dict[celebrity_name] = []

    for entry in os.scandir(img_dir):
        roi_color = get_croped_face(entry.path)
        if isinstance(roi_color, type(None)):
            print("No face detected in image:", entry.path)
            continue
        else:
            cropped_folder = path_to_cr_data + '\\' + celebrity_name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_img_dir.append(cropped_folder)
                print("Created directory:", cropped_folder)

            cropped_file_name = celebrity_name + str(count) + '.jpg'
            cropped_file_path = cropped_folder + '\\' + cropped_file_name

            cv2.imwrite(cropped_file_path, roi_color)
            celebrity_file_names_dict[celebrity_name].append(cropped_file_path)
            count += 1


In [ ]:
def w2d(img, mode='haar', level=1):
    imArray = img
    imArray = cv2.cvtColor(imArray, cv2.COLOR_BGR2GRAY)
    imArray = np.float32(imArray)
    imArray /= 255;

    coeffs = pywt.wavedec2(imArray, mode, level=level)

    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0;

    imArray_H = pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H = np.uint8(imArray_H)

    return imArray_H

In [ ]:
class_dict = {}
count = 0
for celebrity_name in celebrity_file_names_dict.keys():
    class_dict[celebrity_name] = count
    count += 1
class_dict

In [ ]:
X = []
Y = []

for celebrity_name, training_files in celebrity_file_names_dict.items():
    for training_image in training_files:
        img = cv2.imread(training_image)
        if img is None:
            continue
        scaled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(scaled_raw_img, 'db1', 5)  # Fixed: changed scaled_img to scaled_raw_img
        scaled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scaled_raw_img.reshape(32*32*3,1), scaled_img_har.reshape(32*32,1)))
        X.append(combined_img)
        Y.append(class_dict[celebrity_name])


In [ ]:
print(f"Length of X: {len(X[0])}")

In [ ]:
X[0]

In [ ]:
X = np.array(X).reshape(len(X), 4096).astype(float)
X.shape

In [ ]:
X[0]

* Data Cleaning is done, now we will train

#### we will use SVM with rbf kernel tuned with  heuristic finetuning

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel='rbf', C=10))])
pipe.fit(X_train, Y_train)
pipe.score(X_test, Y_test)

In [ ]:
print(classification_report(Y_test, pipe.predict(X_test)))

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
model_parameters = {
    'svm': {
        'model': svm.SVC(gamma='auto', probability=True),
        'params': {
            'svc__C': [1, 10, 100], 
            'svc__kernel': ['linear', 'rbf', 'poly']  
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'randomforestclassifier__n_estimators': [1, 5, 10]  
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'logisticregression__C': [1, 5, 10]  
        }
    }
}

In [ ]:
scores = []
best_estimator = {}
import pandas as pd
for algo, mp in model_parameters.items():
    pipe = make_pipeline(StandardScaler(), mp['model'])
    clf = GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, Y_train)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimator[algo] = clf.best_estimator_

df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

In [ ]:
best_estimator['svm'].score(X_test, Y_test)

In [ ]:
best_estimator['random_forest'].score(X_test, Y_test)

In [ ]:
best_estimator['logistic_regression'].score(X_test, Y_test)

In [ ]:
best_clf = best_estimator['svm']

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, best_clf.predict(X_test))
cm

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
!pip install joblib
import joblib

joblib.dump(best_clf, 'saved_model.pkl')

In [ ]:
import json as js
with open('class_dictionary.json', 'w') as f:
    f.write(js.dumps(class_dict))